# Arboles de Decisión


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from data_builder import load_test_data, load_dev_data, load_small_dev_data


df, target = load_small_dev_data()


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [2]:
from transformers import extractor



extractor.fit(df)
extractor.transform(df)


array([[ u'from: "edwina dotson" <iofigcodgojmqs#####>\r\nto: <paliourg#####>,\r\n\t<sper#####>,\r\n\t<vangelis#####>,\r\n\t<vasvir#####>\r\nsubject:  why?\r\ndate: mon, 29 mar 2004 23:03:23 +0300\r\nmime-version: 1.0\r\ncontent-type: text/html;\r\n\tcharset="iso-8859-7"\r\ncontent-transfer-encoding: quoted-printable\r\nx-priority: 3\r\nx-msmail-priority: normal\r\nx-mimeole: produced by microsoft mimeole v6.00.2900.2527\r\n\r\n<!doctype html public "-//w3c//dtd html 3.2//en">\r\n<html>\r\n<head>\r\n<meta http-equiv=3d"content-type" content=3d"text/html; =\r\ncharset=3diso-8859-7">\r\n<meta name=3d"generator" content=3d"ms exchange server version =\r\n6.5.7036.0">\r\n<title> . why?</title>\r\n</head>\r\n<body>\r\n<!-- converted from text/plain format -->\r\n\r\n<p><font size=3d2>only real super viagdra called ciadlis is =\r\neffective!<br>\r\n&nbsp;annual sale: only $3 per dose<br>\r\ncheck out our website with disscounts and get your fdree bonus =\r\npillls<br>\r\n<a =\r\nhref=3d"http


Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [3]:
from search import find_best_classifier
from sklearn.tree import DecisionTreeClassifier

options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

X = df._get_numeric_data().values
y = target
classifier = find_best_classifier(
    X, y, DecisionTreeClassifier, param_distributions=options, 
    scoring='roc_auc', cv=10, n_jobs=6, n_iter=10)    

Buscando parámetros para DecisionTreeClassifier
Mejor combinación: {'min_samples_split': 92, 'splitter': 'best', 'criterion': 'gini', 'max_depth': 26, 'max_features': 0.50000000000000011}
Mejor valor: 0.977309522806




In [4]:
from helpers import get_scores

get_scores(classifier)

,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
DecisionTreeClassifier,0.933525,0.936222,0.93642,0.939333,0.936222
